In [1]:
import logging

# Configurations
from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
# System components
from llm.azure_openai import AzureOpenAILLM
from embedding.local_embedding import LocalEmbeddingModel
from embedding.request_embedding import RequestEmbeddingModel
# Prompt
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
# Memory
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory_topic import AbstractMemoryTopic
from base_classes.memory.datatypes.data_item import PromptDataItem

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
    handlers=[logging.StreamHandler()]
)


c:\Users\ADMIN\miniconda3\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = AzureOpenAILLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 1.2. Embedding test

### 1.2.1. Request embedding model test

In [ ]:
all_mini_v2_configuration = APIEmbeddingModelConfiguration()
all_mini_v2_configuration.load("configuration/yaml/embedding/all-MiniLM-L6-v2.yaml")
all_mini_v2 = RequestEmbeddingModel(all_mini_v2_configuration)

In [ ]:
all_mini_v2.encode("Hello world")

In [ ]:
all_mini_v2 = RequestEmbeddingModel(all_mini_v2)

In [ ]:
a = {"b": 1, "c": 2}
list(a.keys())

## 1.3. Tool test

### 1.3.1. Python code runner tool

In [ ]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

# 2. Memory test

In [2]:
prompt_data_1 = [{
    'role': 'user',
    'content': 'Calculate the sum of 11 and 20.'
}]
mem_atom_1 = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(prompt_data_1)))

prompt_data_2 = [{
    'role': 'assistant',
    'content': 'In order to calculate the sum of two numbers, we need to do the following:\n\t1. Write a Python code with add_sum() function, receiving 2 variables.\n\t2. Execute the code with the given variables.\n\t3. Return the result of the sum.'
}]
mem_atom_2 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_2)))

prompt_data_3 = [{
    'role': 'assistant',
    'content': 'Here is the Python code to calculate the sum of two numbers:\n\na = 11\nb = 20\nresult = a + b\n\nprint(f"The sum of {a} and {b} is {result}.")'
}]
mem_atom_3 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_3)))

prompt_data_4 = [{
    'role': 'assistant',
    'content': 'After executing the code, we got the sum of 11 and 20 is 31.'
}]
mem_atom_4 = AbstractMemoryAtom(data=PromptDataItem(AssistantMessagePrompt(prompt_data_4)))

2025-05-09 23:11:21,469		DEBUG	[🔧UserMessagePrompt] Prompt initialized with 1 messages.
2025-05-09 23:11:21,470		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 7015ded0-4d42-45c7-8194-53aff81614ff | Memory Atom type: 1; Memory Atom state: 1
2025-05-09 23:11:21,471		DEBUG	[🔧AssistantMessagePrompt] Prompt initialized with 1 messages.
2025-05-09 23:11:21,472		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 2d91f323-3474-4ef0-bd38-38ef903faeb3 | Memory Atom type: 2; Memory Atom state: 1
2025-05-09 23:11:21,473		DEBUG	[🔧AssistantMessagePrompt] Prompt initialized with 1 messages.
2025-05-09 23:11:21,473		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 010fe044-22cf-4e91-b3b9-6739a169338e | Memory Atom type: 2; Memory Atom state: 1
2025-05-09 23:11:21,474		DEBUG	[🔧AssistantMessagePrompt] Prompt initialized with 1 messages.
2025-05-09 23:11:21,475		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 2b2a271f-dfdd-4ceb-8aac-8ccb83b2aca1 | Memory Atom type: 2; Memory Atom state: 1


In [3]:
mem_block = AbstractMemoryBlock()
mem_block.add_memory_atom(mem_atom_1)
mem_block.add_memory_atom(mem_atom_2)
mem_block.add_memory_atom(mem_atom_3)
mem_block.add_memory_atom(mem_atom_4)

mem_block.mem_atom_graph = {
        mem_atom_1.mem_atom_id: [mem_atom_2.mem_atom_id],
        mem_atom_2.mem_atom_id: [mem_atom_3.mem_atom_id, mem_atom_4.mem_atom_id],
        mem_atom_3.mem_atom_id: [],
        mem_atom_4.mem_atom_id: []
    }

2025-05-09 23:11:24,182		DEBUG	[🔧AbstractMemoryBlock] Memory Block ID: b8569875-2661-4b9e-a2ae-06449747470d | Memory Block state: 1
2025-05-09 23:11:24,182		DEBUG	[🔧AbstractMemoryBlock] Memory Block ID: b8569875-2661-4b9e-a2ae-06449747470d | Memory Block state: 2
2025-05-09 23:11:24,184		DEBUG	[🔧AbstractMemoryBlock] Memory Block ID: b8569875-2661-4b9e-a2ae-06449747470d | Memory Block state: 2
2025-05-09 23:11:24,184		DEBUG	[🔧AbstractMemoryBlock] Memory Block ID: b8569875-2661-4b9e-a2ae-06449747470d | Memory Block state: 2


In [4]:
mem_topic = AbstractMemoryTopic()
mem_topic.insert_mem_block(mem_block)

2025-05-09 23:11:26,790		DEBUG	[🔧AbstractMemoryTopic] Inserted Memory Block ID b8569875-2661-4b9e-a2ae-06449747470d into Memory Topic 599fd26c-cbd9-4050-819c-74145b91ce16. New memBlock's topic conatiner ID: 599fd26c-cbd9-4050-819c-74145b91ce16.
